In [1]:
import copy
import importlib
import os
import pathlib
import shutil

import prompt
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import (
    WatsonxLLM,
)

from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams


load_dotenv()


model = Model(
    model_id=ModelTypes.FLAN_T5_XXL,
    credentials={
        "apikey": os.getenv("IBM_API_KEY"),
        "url": "https://us-south.ml.cloud.ibm.com",
    },
    params={
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MAX_NEW_TOKENS: 100,
        GenParams.TEMPERATURE: 0,
        GenParams.RANDOM_SEED: 12345,
    },
    project_id=os.getenv("PROJECT_ID"),
)

llm = WatsonxLLM(model=model)

print(llm("hello how are you?"))

i am fine


In [18]:
def build_prompt(question):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    BOS, EOS = "<s>", "</s>"
    DEFAULT_SYSTEM_PROMPT = """
    You are an expert Q&A system that is trusted around the world. Always answer the query using the provided context information, and not prior knowledge.
    Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
    """
    prompt = "".join([B_SYS, DEFAULT_SYSTEM_PROMPT, E_SYS, question]).strip()
    return f"{BOS}{B_INST} {prompt} {E_INST}"

ROUTING_TEMPLATE = """
Classify the following question into one of these options. 

1) Property specific transactional query
2) General transactional query
3) Property specific general query
4) General query

Query: Still got available units for Residensi Zig?

Answer: 1

Query: what is a good house that is less then 900k?

Answer: 2

Query: any property in Mont Kiara?

Answer: 3

Query: what are the amenities in the minh?

Answer: 3

Query: What are the properties that have a price range from RM300,000 to RM600,000?

Answer: 2

Question: {{question}} 

Answer:
"""

In [19]:
questions = [
    {"question": "How many units available for The Connaught One?", "answer": 1},
    {"question": "What are the layout types for The MINH?", "answer": 1},


    {"question": "What properties have a price range of RM400,000 to RM700,000?", "answer": 2},


    {"question": "What are the nearby amenities for The Connaught One?", "answer": 3},
    {"question": "Is The Connaught One near the MRT station?", "answer": 3},
    {"question": "How many blocks are there for The MINH?", "answer": 3},


    {"question": "How much for non bumi puteri?", "answer": 4},
    {"question": "How many parking lots?", "answer": 4},
    {"question": "I'm interested in the MINH.", "answer": 4},


]

generated_answers = [
    llm(ROUTING_TEMPLATE.replace("{{question}}", x["question"])) for x in questions
]

df = pd.DataFrame(questions)
df["generated_answer"] = generated_answers
df["generated_answer"] = df["generated_answer"].astype(int)
print("Accuracy: {:.2f}".format((df["answer"] == df["generated_answer"]).sum() / df.shape[0]))
df

Accuracy: 0.56


,question,answer,generated_answer
0,How many units available for The Connaught One?,1,1
1,What are the layout types for The MINH?,1,3
2,"What properties have a price range of RM400,00...",2,2
3,What are the nearby amenities for The Connaugh...,3,3
4,Is The Connaught One near the MRT station?,3,3
5,How many blocks are there for The MINH?,3,3
6,How much for non bumi puteri?,4,1
7,How many parking lots?,4,3
8,I'm interested in the MINH.,4,3
